# Notebook 02: Data Cleaning

## Purpose
This notebook focuses on improving data quality by:
- Handling missing values using **multiple techniques** (as required)
- Removing duplicates
- Fixing data type inconsistencies
- Preparing clean data for exploratory analysis

## Learning Objectives
- Apply ALL required missing value treatment methods
- Understand when each method is appropriate
- Document assumptions and reasoning for data cleaning decisions

---
## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df_original = pd.read_csv('../data/MinoAI_dataset.csv')

print(f"Dataset loaded: {df_original.shape[0]:,} rows, {df_original.shape[1]} columns")
print(f"\nMemory usage: {df_original.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

---
## 2. Identify Missing Values

Before cleaning, we need to understand the extent and pattern of missing data.

In [ ]:
# Calculate missing values
missing_data = pd.DataFrame({
    'Column': df_original.columns,
    'Missing_Count': df_original.isnull().sum().values,
    'Missing_Percentage': (df_original.isnull().sum().values / len(df_original) * 100)
})

# Filter columns with missing values
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("COLUMNS WITH MISSING VALUES:")
print("="*80)
print(missing_data.to_string(index=False))

---
## 3. Missing Value Treatment Methods

As required by the assignment, we will apply **ALL** of the following methods:
1. **Forward Fill** - Propagate last valid observation forward
2. **Backward Fill** - Use next valid observation to fill gap
3. **Interpolation** - Estimate values based on surrounding data
4. **Mean/Median Imputation** - Fill with statistical measures

We'll create separate DataFrames for each method to compare results.

### Assumptions:
- **Forward fill** is suitable for time-series data where values tend to persist
- **Backward fill** is useful when future values are known and relevant
- **Interpolation** works well for numerical data with linear relationships
- **Mean imputation** is appropriate for normally distributed data without outliers

### 3.1 Method 1: Forward Fill

**Explanation**: Forward fill propagates the last valid observation forward to fill missing values. This is particularly useful for time-series data or when we assume values remain constant until changed.

**When to use**: Best for sequential data where the previous value is a reasonable estimate.

In [ ]:
# Create a copy for forward fill
df_ffill = df_original.copy()

# Apply forward fill
df_ffill = df_ffill.fillna(method='ffill')

# Check remaining missing values
print("FORWARD FILL RESULTS:")
print("="*80)
print(f"Remaining missing values: {df_ffill.isnull().sum().sum()}")
print(f"\nMissing values by column:")
print(df_ffill.isnull().sum()[df_ffill.isnull().sum() > 0])

### 3.2 Method 2: Backward Fill

**Explanation**: Backward fill uses the next valid observation to fill missing values. This is the reverse of forward fill.

**When to use**: Useful when future values are known and can inform past missing values.

In [ ]:
# Create a copy for backward fill
df_bfill = df_original.copy()

# Apply backward fill
df_bfill = df_bfill.fillna(method='bfill')

# Check remaining missing values
print("BACKWARD FILL RESULTS:")
print("="*80)
print(f"Remaining missing values: {df_bfill.isnull().sum().sum()}")
print(f"\nMissing values by column:")
print(df_bfill.isnull().sum()[df_bfill.isnull().sum() > 0])

### 3.3 Method 3: Interpolation

**Explanation**: Interpolation estimates missing values based on the values before and after the gap. It assumes a linear relationship between data points.

**When to use**: Best for numerical data with continuous values and linear trends.

In [ ]:
# Create a copy for interpolation
df_interpolate = df_original.copy()

# Apply interpolation to numerical columns only
numerical_cols = df_interpolate.select_dtypes(include=['int64', 'float64']).columns
df_interpolate[numerical_cols] = df_interpolate[numerical_cols].interpolate(method='linear', limit_direction='both')

# For categorical columns, use forward fill as fallback
df_interpolate = df_interpolate.fillna(method='ffill').fillna(method='bfill')

# Check remaining missing values
print("INTERPOLATION RESULTS:")
print("="*80)
print(f"Remaining missing values: {df_interpolate.isnull().sum().sum()}")
print(f"\nMissing values by column:")
print(df_interpolate.isnull().sum()[df_interpolate.isnull().sum() > 0])

### 3.4 Method 4: Mean/Median Imputation

**Explanation**: This method replaces missing values with the mean (for normally distributed data) or median (for skewed data) of the column.

**When to use**: 
- **Mean**: For normally distributed numerical data
- **Median**: For skewed data or data with outliers

In [ ]:
# Create a copy for mean/median imputation
df_impute = df_original.copy()

# For numerical columns, use median (more robust to outliers)
numerical_cols = df_impute.select_dtypes(include=['int64', 'float64']).columns

for col in numerical_cols:
    if df_impute[col].isnull().sum() > 0:
        median_value = df_impute[col].median()
        df_impute[col].fillna(median_value, inplace=True)
        print(f"Filled {col} with median: {median_value}")

# For categorical columns, use mode (most frequent value)
categorical_cols = df_impute.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if df_impute[col].isnull().sum() > 0:
        mode_value = df_impute[col].mode()[0] if not df_impute[col].mode().empty else 'Unknown'
        df_impute[col].fillna(mode_value, inplace=True)
        print(f"Filled {col} with mode: {mode_value}")

# Check remaining missing values
print("\nMEAN/MEDIAN IMPUTATION RESULTS:")
print("="*80)
print(f"Remaining missing values: {df_impute.isnull().sum().sum()}")

---
## 4. Compare Methods

Let's compare the effectiveness of each method.

In [ ]:
# Create comparison summary
comparison = pd.DataFrame({
    'Method': ['Original', 'Forward Fill', 'Backward Fill', 'Interpolation', 'Mean/Median Imputation'],
    'Missing_Values': [
        df_original.isnull().sum().sum(),
        df_ffill.isnull().sum().sum(),
        df_bfill.isnull().sum().sum(),
        df_interpolate.isnull().sum().sum(),
        df_impute.isnull().sum().sum()
    ]
})

comparison['Percentage_Complete'] = ((df_original.shape[0] * df_original.shape[1] - comparison['Missing_Values']) / 
                                      (df_original.shape[0] * df_original.shape[1]) * 100)

print("METHOD COMPARISON:")
print("="*80)
print(comparison.to_string(index=False))

### Visualize Comparison

In [ ]:
# Visualize method comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Missing values by method
ax1.bar(comparison['Method'], comparison['Missing_Values'], color='coral')
ax1.set_xlabel('Method', fontsize=11)
ax1.set_ylabel('Missing Values Count', fontsize=11)
ax1.set_title('Missing Values by Imputation Method', fontsize=13, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(comparison['Missing_Values']):
    ax1.text(i, v + 500, str(v), ha='center', va='bottom', fontweight='bold')

# Plot 2: Percentage complete
ax2.bar(comparison['Method'], comparison['Percentage_Complete'], color='lightgreen')
ax2.set_xlabel('Method', fontsize=11)
ax2.set_ylabel('Percentage Complete (%)', fontsize=11)
ax2.set_title('Data Completeness by Method', fontsize=13, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.set_ylim([95, 100])
ax2.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(comparison['Percentage_Complete']):
    ax2.text(i, v + 0.1, f'{v:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

---
## 5. Select Best Method and Create Clean Dataset

### Decision Rationale:

Based on the comparison above, we will use **Mean/Median Imputation** as our primary cleaning method because:

1. It completely eliminates missing values
2. It's statistically sound for this dataset
3. It doesn't introduce bias from sequential filling
4. It's appropriate for the type of data we have (property listings)

**Assumption**: Missing reviews data likely indicates listings with no reviews, so using median (0 or low values) is reasonable.

In [ ]:
# Use the imputed dataset as our clean dataset
df_clean = df_impute.copy()

print("Selected method: Mean/Median Imputation")
print(f"Clean dataset shape: {df_clean.shape}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")

---
## 6. Check for Duplicates

In [ ]:
# Check for duplicate rows
duplicates = df_clean.duplicated().sum()

print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    print(f"\nRemoving {duplicates} duplicate rows...")
    df_clean = df_clean.drop_duplicates()
    print(f"New shape: {df_clean.shape}")
else:
    print("No duplicates found!")

# Check for duplicate IDs (should be unique)
duplicate_ids = df_clean['id'].duplicated().sum()
print(f"\nDuplicate IDs: {duplicate_ids}")

---
## 7. Fix Data Types

Some columns may need data type conversion for proper analysis.

In [ ]:
# Check current data types
print("CURRENT DATA TYPES:")
print("="*80)
print(df_clean.dtypes)

In [ ]:
# Convert last_review to datetime
if 'last_review' in df_clean.columns:
    # Convert to datetime, handling errors
    df_clean['last_review'] = pd.to_datetime(df_clean['last_review'], errors='coerce')
    print("Converted 'last_review' to datetime")

# Ensure numerical columns are correct type
numerical_columns = ['price', 'minimum_nights', 'number_of_reviews', 
                     'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

for col in numerical_columns:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print("\nUPDATED DATA TYPES:")
print("="*80)
print(df_clean.dtypes)

---
## 8. Final Data Quality Check

In [ ]:
# Comprehensive quality check
print("FINAL DATA QUALITY REPORT:")
print("="*80)
print(f"Total Rows: {df_clean.shape[0]:,}")
print(f"Total Columns: {df_clean.shape[1]}")
print(f"Missing Values: {df_clean.isnull().sum().sum()}")
print(f"Duplicate Rows: {df_clean.duplicated().sum()}")
print(f"Memory Usage: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nData Types Summary:")
print(df_clean.dtypes.value_counts())

---
## 9. Save Cleaned Dataset

We'll save the cleaned dataset for use in subsequent notebooks.

In [ ]:
# Save cleaned dataset
df_clean.to_csv('../data/cleaned_dataset.csv', index=False)

print("Cleaned dataset saved to: ../data/cleaned_dataset.csv")
print(f"File size: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

---
## 10. Summary of Data Cleaning Process

### What We Accomplished:

1. **Missing Value Treatment** ✅
   - Applied **Forward Fill**: Propagated last valid values forward
   - Applied **Backward Fill**: Used next valid values to fill gaps
   - Applied **Interpolation**: Estimated values based on linear relationships
   - Applied **Mean/Median Imputation**: Filled with statistical measures
   - **Selected**: Mean/Median Imputation as the best method

2. **Duplicate Removal** ✅
   - Checked for and removed duplicate rows
   - Verified ID uniqueness

3. **Data Type Fixes** ✅
   - Converted `last_review` to datetime format
   - Ensured numerical columns have correct types

4. **Quality Assurance** ✅
   - Verified no missing values remain
   - Confirmed data integrity
   - Saved clean dataset for analysis

### Key Assumptions Made:

- Missing review data indicates listings with no reviews (median imputation appropriate)
- Median is preferred over mean due to potential outliers in price and other metrics
- Mode is appropriate for categorical missing values
- All listings should have unique IDs

### Next Steps:

The cleaned dataset is now ready for **Exploratory Data Analysis (EDA)** in the next notebook, where we will:
- Visualize distributions
- Analyze relationships between variables
- Identify patterns and trends
- Detect outliers

---
**Next Notebook**: [03_exploratory_data_analysis.ipynb](03_exploratory_data_analysis.ipynb)